# RAG for CSV Data

In [ ]:
#Install Libraries
# for reading csv file 
#for similarity search 
#for arrays 
#openai 
FIX ME

In [ ]:
import pandas as pd
import faiss
import numpy as np
from openai import OpenAI

In [ ]:
#store data file in variable 
FIXME
df = FIXME
df

In [ ]:
def generate_text_representation(row):
    author_name = row["Author's Name"]
    title = row["Title"]
    publisher = row["Publisher / Place"]
    year = row["Year"]
    return f"{author_name} - {title} - {publisher} - {year}"

df["text_representation"] = df.apply(FIXME, axis=1)


In [ ]:
df["text_representation"]

In [ ]:
client = OpenAI(
    api_key="enter your API key here",
    base_url="https://integrate.api.nvidia.com/v1"
)

In [ ]:
embeddings = []
for text in df["FIXME"].tolist():
    response = client.embeddings.create(
        input=[text],
        model="nvidia/nv-embedqa-e5-v5",
        encoding_format="float",
        extra_body={"input_type": "query", "truncate": "NONE"}
    )
    embeddings.append(response.data[0].embedding)


In [ ]:
embedding_dim = len(embeddings[0])
embedding_array = np.array(embeddings).astype('float32')
embedding_dim = embedding_array.shape[1]
index = faiss.IndexFlatL2(embedding_dim)
index.add(embedding_array)

In [ ]:
metadata_file = "metadata.npy"
index_file = "faiss_index"

In [ ]:
np.save(metadata_file, df["text_representation"].tolist())
faiss.write_index(index, index_file)

In [ ]:
def search_faiss(query, top_k=2):
    query_response = client.embeddings.create(
        input=[query],
        model="FIXME",
        encoding_format="FIXME",
        extra_body={"input_type": "query", "truncate": "NONE"}
    )
    query_embedding = np.array([query_response.data[0].embedding]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    metadata = np.load(metadata_file, allow_pickle=True)
    results = [(metadata[idx], distances[0][i]) for i, idx in enumerate(indices[0])]
    return results


In [ ]:
query = "Data Structures"
results = search_faiss(query)
print(f"Search Results for '{query}':")
for result, score in results:
    print(f"- {result} (Distance: {score:.2f})")

In [ ]:
def beautify_results(results):
    beautified_output = []
    unique_books = {}

    for result, score in results:
        if result not in unique_books:
            unique_books[result] = {"info": result, "distance": score}

    for book in unique_books.values():
        book_info = book["info"]
        description = summarize_book(book_info)
        beautified_output.append({
            "book": book_info.split(' - ')[1],   
            "author": book_info.split(' - ')[0],
            "publisher": book_info.split(' - ')[2],
            "year": book_info.split(' - ')[3],
            "description": description
        })
    
    return beautified_output


In [ ]:
def summarize_book(book_info):
    prompt = f"Provide a concise and clear description for the following book: {book_info}. Avoid introductory phrases like 'Here is a concise description of the book' and focus on the core content."
    response = client.chat.completions.create(
        model="meta/llama-3.1-8b-instruct",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        top_p=0.7,
        max_tokens=100,
        stream=False,
    )
    
    description = response.choices[0].message.content.strip()

    unwanted_phrases = [
        "Here is a concise description of the book:",
        "Here is a brief summary of the book:",
        "This is a brief description of the book:",
        "This book,"
    ]
    for phrase in unwanted_phrases:
        if description.startswith(phrase):
            description = description.replace(phrase, "").strip()
    
    return description


In [ ]:
from IPython.display import display, Markdown

def format_results(beautified_results):
    for book in beautified_results:
     
        formatted_result = f"""
---
**Book:** {book['book']} <br>
**Author:** {book['author']} <br>
**Publisher:** {book['publisher']} <br>
**Year:** {book['year']} <br>
**Description:** {book['description']} <br>
"""

        display(Markdown(formatted_result))

In [ ]:
def summarize_results(beautified_results):

    output = []
    for idx, book in enumerate(beautified_results, start=1):
        output.append(f"{book['book']} by {book['author']}")

    return "Hello VIPS Trainees, I got these results:\n" + "\n".join(output)

In [ ]:
query = "organic chemistry"
results = search_faiss(query)

In [ ]:
beautified_results = beautify_results(results)

In [ ]:
print(f"Search Results for '{query}':\n")
format_results(beautified_results)
summary = summarize_results(beautified_results)
print(summary)

In [ ]:
!pip install -U nvidia-riva-client

In [ ]:
!git clone https://github.com/nvidia-riva/python-clients.git

In [ ]:
import subprocess


command = [
    "python", "python-clients/scripts/tts/talk.py",
    "--server", "grpc.nvcf.nvidia.com:443", "--use-ssl",
    "--metadata", "function-id", "877104f7-e885-42b9-8de8-f6e4c6303969",
    "--metadata", "authorization", f"Bearer $NIM_API_KEY", # Enter your api key
    "--language-code", "en-US",
    "--text", summary,
    "--voice", "Magpie-Multilingual.EN-US.Aria",
    "--output", "audio.wav"
]

try:
    result = subprocess.run(command, capture_output=True, text=True, check=True)
    print("Output:", result.stdout)
except subprocess.CalledProcessError as e:
    print("Error occurred:", e.stderr)


In [ ]:
from IPython.display import Audio

file_path = "audio.wav"
Audio(file_path, autoplay=True)